In [4]:
"""
Script de procesado de agentes EBA.
===================================
Este script carga un archivo JSON con información de agentes de pago (PSD_AG),
expande la columna "Properties", filtra por determinados criterios,
normaliza algunas columnas y cruza los datos con listados de municipios
españoles y códigos de comunidades autónomas.

"""

# -----------------------------
# 1. Importación de librerías
# -----------------------------
import tkinter as tk                 # GUI minimal para diálogos de selección de archivos
from tkinter import filedialog       # Diálogo de archivos de Tkinter
import json                          # Manejo de archivos JSON
import pandas as pd                  # Manipulación de datos en DataFrames
import re                            # Expresiones regulares
import openpyxl                      # Lectura/escritura de archivos Excel
from fuzzywuzzy import fuzz          # Coincidencia difusa (no se usa en la lógica actual)
from fuzzywuzzy import process       # Utilidades adicionales de fuzzywuzzy
from faker import Faker              # Generador de datos falsos (no se usa en la lógica actual)
import random                        # Generador de números aleatorios (no se usa en la lógica actual)
from datetime import datetime, time  # Manejo de fechas y horas

# -----------------------------
# 2. Inicio del procesamiento
# -----------------------------
print("Inicio del script")

# Configuramos una ventana raíz oculta de Tkinter para utilizar sus diálogos
root = tk.Tk()
root.withdraw()  # Oculta la ventana principal para que solo se vea el diálogo

# -----------------------------------------
# 3. Selección y carga del archivo JSON EBA
# -----------------------------------------

ruta_archivo = filedialog.askopenfilename()  # Diálogo para seleccionar el JSON de agentes EBA

if ruta_archivo:
    try:
        with open(ruta_archivo, encoding="utf8") as file:
            data = json.load(file)  # Carga el JSON como estructura Python
            print(data)            # Muestra el contenido (útil para depuración)
    except FileNotFoundError:
        print(f"El archivo '{ruta_archivo}' no fue encontrado.")
    except Exception as e:
        print(f"Ocurrió un error: {e}")
else:
    print("No se seleccionó ningún archivo.")

# --------------------------------------------------------
# 4. Conversión del JSON a DataFrame y filtrado de agentes
# --------------------------------------------------------

pagos = pd.DataFrame(data[1])  # Se asume que la lista en data[1] contiene los registros

# Filtrar solo entidades de tipo PSD_AG (agentes)
agentes = pagos[pagos['EntityType'] == 'PSD_AG']

# ------------------------------------------------------------------------
# 5. Función para expandir el campo "Properties" en columnas individuales
# ------------------------------------------------------------------------

def expand_properties(df):
    """Expande la columna 'Properties' en columnas separadas.

    Cada elemento de 'Properties' es una lista de diccionarios. La función
    crea un DataFrame auxiliar con la unión de todos esos diccionarios y lo
    concatena al DataFrame original (sin la columna 'Properties').
    """
    properties_data = []
    for _, row in df.iterrows():
        prop_dict = {}
        if isinstance(row['Properties'], list):
            for prop in row['Properties']:
                if isinstance(prop, dict):
                    prop_dict.update(prop)  # Une los pares clave‑valor
        properties_data.append(prop_dict)

    properties_df = pd.DataFrame(properties_data)

    # Sustituye la columna original por las nuevas columnas expandidas
    df = pd.concat([
        df.drop('Properties', axis=1).reset_index(drop=True),
        properties_df.reset_index(drop=True)
    ], axis=1)
    return df

# Aplicamos la expansión si existe al menos un registro
if not agentes.empty:
    agentes_expandido = expand_properties(agentes)
    print(agentes_expandido.head())  # Vista rápida del resultado
else:
    print("El DataFrame está vacío, no se pueden expandir las propiedades.")

# --------------------------------------------------------------
# 6. Filtrado por columnas específicas en bloques (chunks) grandes
# --------------------------------------------------------------

chunk_size = 10000  # Procesamos en trozos para limitar uso de memoria
filtered_chunks = []  # Almacenamos los trozos que cumplen la condición

for i in range(0, len(agentes_expandido), chunk_size):
    chunk = agentes_expandido.iloc[i:i + chunk_size]
    # Condiciones de filtrado
    filtered_chunk = chunk[
        (chunk['ENT_COU_RES'] == 'ES') &      # País de residencia = España
        (chunk['ENT_TYP_PAR_ENT'] == 'PSD_PI') &  # Tipo de entidad parental = PSD_PI
        (chunk['DER_CHI_ENT_AUT'] == 'Active')    # Estado de autorización = Activo
    ]
    filtered_chunks.append(filtered_chunk)

# Unión de todos los trozos filtrados
agentes_filtrado = pd.concat(filtered_chunks)

# --------------------------------------------------
# 7. Eliminación de columnas irrelevantes o repetidas
# --------------------------------------------------

columnas_a_eliminar = [
    "CA_OwnerID",
    "EntityCode",
    "EntityType",
    "Services",
    "__EBA_EntityVersion",
    "DER_CHI_ENT_AUT",
    "ENT_COU_RES",
    "ENT_TYP_PAR_ENT"
]

agentes_filtrado = agentes_filtrado.drop(columns=columnas_a_eliminar)

# ----------------------------------------------------------------
# 8. Normalización de valores de ciudad y código postal en el DataFrame
# ----------------------------------------------------------------

def separar_letras_numeros(df):
    """Separa letras y números en 'ENT_TOW_CIT_RES' y rellena 'ENT_POS_COD'."""
    for i, row in df.iterrows():
        text = ''.join(re.findall(r'[^\d]+', str(row['ENT_TOW_CIT_RES'])))  # Solo letras
        numbers = ''.join(re.findall(r'\d+', str(row['ENT_TOW_CIT_RES'])))  # Solo números
        df.at[i, 'ENT_TOW_CIT_RES'] = text  # Reemplaza con solo letras
        # Solo se actualiza ENT_POS_COD si está vacío o no es numérico
        if numbers and not str(row['ENT_POS_COD']).isdigit():
            df.at[i, 'ENT_POS_COD'] = numbers
        elif not str(row['ENT_POS_COD']).isdigit():
            df.at[i, 'ENT_POS_COD'] = ''
    return df

agentes_c = separar_letras_numeros(agentes_filtrado)

# Pone la columna de ciudades en formato Title Case
def title_case(s):
    return ' '.join(word.capitalize() for word in s.split())

agentes_c['ENT_TOW_CIT_RES'] = agentes_c['ENT_TOW_CIT_RES'].apply(title_case)

# ------------------------------------------------------------
# 9. Carga de listado de municipios españoles desde CSV
# ------------------------------------------------------------

root = tk.Tk()
root.withdraw()  # Oculta la ventana de Tkinter para otro diálogo

ruta_archivo = filedialog.askopenfilename()  # Selección del CSV de municipios

municipios_df = None  # Inicializamos

if ruta_archivo:
    try:
        municipios_df = pd.read_csv(ruta_archivo, encoding='utf-8-sig', sep=';')
    except FileNotFoundError:
        print(f"El archivo '{ruta_archivo}' no fue encontrado.")
    except Exception as e:
        print(f"Ocurrió un error: {e}")
else:
    print("No se seleccionó ningún archivo.")

# Limpieza de índice (por si el CSV ya viene con uno)
municipios_dfs = municipios_df.reset_index(drop=True)
municipios_df1 = municipios_dfs.dropna(how='all')  # Elimina filas completamente vacías

# ------------------------------------------------------------
# 10. Carga del CSV con correspondencia de CCAA ↔ Código postal
# ------------------------------------------------------------

print(openpyxl.__version__)  # Muestra versión de openpyxl (debug)

root = tk.Tk()
root.withdraw()

ruta_archivo = filedialog.askopenfilename()  # Selección del CSV de CCAA

CCAA_df = None

if ruta_archivo:
    try:
        CCAA_df = pd.read_csv(ruta_archivo, encoding='utf-8-sig', sep=';')
        CCAA_df1 = CCAA_df.reset_index(drop=True).dropna(how='all')

        if not CCAA_df1.empty:
            print(CCAA_df1)  # Vista previa
        else:
            print("El archivo CSV está vacío o no contiene datos válidos.")

    except FileNotFoundError:
        print(f"Error: El archivo '{ruta_archivo}' no fue encontrado.")
    except pd.errors.ParserError:
        print(f"Error: No se pudo analizar el archivo CSV. Verifica el formato.")
    except Exception as e:
        print(f"Error inesperado: {e}")
else:
    print("No se seleccionó ningún archivo.")

# --------------------------------------------------------------------
# 11. Cruce entre municipios y códigos de CCAA a partir del código postal
# --------------------------------------------------------------------

# 1) Asegura que los CP tengan 5 dígitos con ceros a la izquierda
municipios_df1['Código postal'] = municipios_df1['Código postal'].astype(str).str.zfill(5)

# 2) Extrae los dos primeros dígitos del CP para enlazar con la tabla de CCAA
municipios_df1['Cod_cp_temp'] = municipios_df1['Código postal'].str[:2]

# 3) Convierte a string y rellena con ceros la columna de códigos en CCAA
CCAA_df1['Cod_CP'] = CCAA_df1['Cod_CP'].astype(str).str.zfill(2)

# 4) Realiza el merge entre ambos DataFrames
municipios_df1 = pd.merge(
    municipios_df1,
    CCAA_df1,
    left_on='Cod_cp_temp',
    right_on='Cod_CP',
    how='left'
)

# 5) Reordena las columnas para un resultado más claro
municipios_df1 = municipios_df1[['Municipio', 'Provincia', 'CCAA2', 'Código postal', 'Cod_cp_temp', 'Cod_CP']]

# 6) Elimina columnas temporales utilizadas durante el merge
municipios_df2 = municipios_df1.drop(['Cod_cp_temp', 'Cod_CP'], axis=1)

# Muestra el DataFrame final resultante
print(municipios_df2)



Inicio del script


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



  CA_OwnerID                          EntityCode EntityType Services  \
0     BG_BNB  PSD_EMI!BG_BNB!205968825!204892794     PSD_AG      NaN   
1     BG_BNB  PSD_EMI!BG_BNB!205968825!205762565     PSD_AG      NaN   
2     BG_BNB  PSD_EMI!BG_BNB!205968825!203093257     PSD_AG      NaN   
3     BG_BNB  PSD_EMI!BG_BNB!205968825!200415137     PSD_AG      NaN   
4     BG_BNB  PSD_EMI!BG_BNB!205968825!206802144     PSD_AG      NaN   

  __EBA_EntityVersion ENT_NAT_REF_COD  \
0   20250321140709487       204892794   
1   20250321140833323       205762565   
2   20250321140958052       203093257   
3   20250321141117316       200415137   
4   20250321141237403       206802144   

                                             ENT_NAM  \
0                     [Пиза - Д ЕООД, Piza - D EOOD]   
1           [Сотиров транс ЕООД, Sotirov trans EOOD]   
2  [Естрея-Емануил Милков ЕООД, Estreya-Emanuil M...   
3          [Шаренкапови 7 ЕООД, Sharenkapovi 7 EOOD]   
4                         [Трантор ООД, 

C:\Users\guada\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3325: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


3.1.3
    Cod_CCAA                         CCAA  Cod_CP               Provincia  \
0          1                    Andalucía       4                 Almería   
1          1                    Andalucía      11                   Cádiz   
2          1                    Andalucía      14                 Córdoba   
3          1                    Andalucía      18                 Granada   
4          1                    Andalucía      21                  Huelva   
5          1                    Andalucía      23                    Jaén   
6          1                    Andalucía      29                  Málaga   
7          1                    Andalucía      41                 Sevilla   
8          2                       Aragón      22                  Huesca   
9          2                       Aragón      44                  Teruel   
10         2                       Aragón      50                Zaragoza   
11         3      Asturias, Principado de      33                Astur

C:\Users\guada\Anaconda3\lib\site-packages\ipykernel_launcher.py:222: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\guada\Anaconda3\lib\site-packages\ipykernel_launcher.py:225: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [7]:
"""
Fuzzy‑matching de nombres de municipio con RapidFuzz
===================================
Este script devuelve los datos de municipios entre dos ficheros.

"""
from __future__ import annotations
# Imports necesarios (pandas ya estaba usado en el bloque principal)
!pip install rapidfuzz
import pandas as pd
from rapidfuzz import fuzz, process  # rapidfuzz ofrece matching difuso rápido


def encontrar_mejores_coincidencias_rapidfuzz_con_cache(
    municipios_data6: pd.Series | list[str],
    lista_municipios: list[str],
    umbral: int = 85,
):
    """Devuelve la mejor coincidencia difusa para cada municipio.

    * **municipios_data6**: iterable con los nombres a buscar.
    * **lista_municipios**: universo de comparación (nombres oficiales).
    * **umbral**: porcentaje mínimo (0‑100) de similitud aceptado.
    * Usa un **cache** interno para no recalcular coincidencias repetidas.
    """
    cache: dict[str, str | None] = {}
    resultados: list[str | None] = []

    for municipio in municipios_data6:
        # Si ya calculamos este municipio, recuperamos el resultado del cache
        if municipio in cache:
            resultados.append(cache[municipio])
            continue

        # Buscamos la coincidencia más parecida en lista_municipios
        mejor_coincidencia, similitud, _ = process.extractOne(
            municipio,
            lista_municipios,
            scorer=fuzz.ratio,  # Métrica de similitud (0‑100)
        )

        # Solo aceptamos la coincidencia si supera el umbral indicado
        if similitud >= umbral:
            cache[municipio] = mejor_coincidencia
        else:
            cache[municipio] = None  # Marcamos como sin coincidencia válida
        resultados.append(cache[municipio])

    return resultados

# ---------------------------------------------------------------------------
# Ejecución del matching y merge con el DataFrame de municipios
# ---------------------------------------------------------------------------

# Convertimos la columna "Municipio" a lista una única vez para evitar
# recalcularla en cada iteración de RapidFuzz.
lista_municipios = municipios_df2["Municipio"].tolist()

# Añadimos la columna con la mejor coincidencia encontrada
agentes_c["Mejor_Coincidencia"] = encontrar_mejores_coincidencias_rapidfuzz_con_cache(
    agentes_c["ENT_TOW_CIT_RES"],
    lista_municipios,
)

# Hacemos un *merge* usando la coincidencia encontrada ↔ municipio oficial
agentes_c_con_municipios = pd.merge(
    agentes_c,
    municipios_df2,
    left_on="Mejor_Coincidencia",
    right_on="Municipio",
    how="left",
)

# Eliminamos columnas auxiliares para dejar el DataFrame final limpio
agentes_c_con_municipios = agentes_c_con_municipios.drop(
    ["Mejor_Coincidencia", "Municipio"],
    axis=1,
)

# Mostramos el resultado (o podrías guardarlo a disco)
print(agentes_c_con_municipios)


                  ENT_NAT_REF_COD                                ENT_NAM  \
0                       Y4785493N                          NIGAR SULTANA   
1                       02395680T          NAOUAL EL KOURACHI EL BACHIRI   
2                       B86261252                          RICTELECOM SL   
3                       B67299792                        BHALOT BROS S L   
4                       Y6487752H                   JAKAIRA HOSSAIN KHAN   
...                           ...                                    ...   
25634                   Y1641002N    JUAN MATOS REYES - AGENTE IN SPAGNA   
25635                   Y2701905V     MEHTAB IFTIKHAR - AGENTE IN SPAGNA   
25636                   Y2838493P    MD KAMRUL  HASAN - AGENTE IN SPAGNA   
25637                   Y3333433B  MONIRUZZAMAN MONIR - AGENTE IN SPAGNA   
25638  PSD_PI!FR_ACPR!49786!51599                          WPS ESPANA SA   

                                                 ENT_ADD  \
0                          

In [8]:
"""
Generador de ficheros CSV con transacciones simuladas
===================================
Este script genera ficheros que simulan transacciones de riesgo y
no riesgo realizados en agentes y sus correspondientes sujetos obligados.

"""


from faker import Faker        # Generador de datos falsos realistas
import random                  # Utilidades de aleatoriedad de la librería estándar
from datetime import datetime, time  # Manejo de fechas y horas
import pandas as pd            # Manipulación tabular
import re                      # Expresiones regulares (no se usan luego, pero lo dejamos)
import numpy as np             # Operaciones numéricas y aleatorias vectorizadas

# ---------------------------------------------------------------------------
# 1. Configuración inicial de Faker y nombres de archivos
# ---------------------------------------------------------------------------

faker = Faker('es_ES')  # Locale español para nombres y NIE

# --- Solicitamos al usuario cuántos CSV debe generar ---
num_ficheros = int(input("Introduce el número de ficheros CSV a generar: "))
# Nombres como Agente_A_Entidad_1.csv, Agente_B_Entidad_2.csv…
ficheros = [f"Agente_{chr(65+i)}_Entidad_{i+1}.csv" for i in range(num_ficheros)]

# ---------------------------------------------------------------------------
# 2. Generación de clientes ordenantes únicos
# ---------------------------------------------------------------------------

num_clientes_ordenantes = int(300 * 0.8)  # 80 % de 300 → 240 clientes
clientes_ordenantes = []
for _ in range(num_clientes_ordenantes):
    pais = faker.country()
    cliente = {
        "NOMBRE_ORDENANTE": faker.first_name(),
        "APELLIDO_ORDENANTE": faker.last_name(),
        "SEGUNDO_APELLIDO_ORDENANTE": faker.last_name() if random.random() > 0.2 else '',
        "PAIS_DOC_ORDENANTE": pais,
        "NUM_DOC_ORDENANTE": faker.nie(),  # Número de documento (NIE)
        "PAIS_NAC_ORDENANTE": faker.country(),
        "FECHA_NAC_ORDENANTE": faker.date_of_birth().strftime('%d/%m/%Y'),
        "es_Agente": False  # Flag que después se usará para marcar al agente designado
    }
    clientes_ordenantes.append(cliente)

# ---------------------------------------------------------------------------
# 3. Beneficiarios basados en los ordenantes
# ---------------------------------------------------------------------------

num_beneficiarios_basados = int(300 * 0.5)  # 50 % de 300 → 150 beneficiarios
beneficiarios_basados = []
for ordenante in clientes_ordenantes:
    apellido_base = ordenante['APELLIDO_ORDENANTE']
    if random.random() < 0.9:  # 90 % comparten apellido / país con ordenante
        beneficiario = {
            "NOMBRE_BENEFICIARIO": faker.first_name(),
            "APELLIDO_BENEFICIARIO": apellido_base,
            "SEGUNDO_APELLIDO_BENEFICIARIO": faker.last_name() if random.random() > 0.2 else '',
            "PAIS_DESTINO": ordenante['PAIS_DOC_ORDENANTE']
        }
    else:  # 10 % no relacionados
        beneficiario = {
            "NOMBRE_BENEFICIARIO": faker.first_name(),
            "APELLIDO_BENEFICIARIO": faker.last_name(),
            "SEGUNDO_APELLIDO_BENEFICIARIO": faker.last_name() if random.random() > 0.2 else '',
            "PAIS_DESTINO": faker.country()
        }
    beneficiarios_basados.append(beneficiario)

# ---------------------------------------------------------------------------
# 4. Beneficiarios completamente aleatorios (20 %)
# ---------------------------------------------------------------------------

num_beneficiarios_aleatorios = int(300 * 0.2)  # 20 % de 300 → 60 beneficiarios
beneficiarios_aleatorios = []
for _ in range(num_beneficiarios_aleatorios):
    beneficiario = {
        "NOMBRE_BENEFICIARIO": faker.first_name(),
        "APELLIDO_BENEFICIARIO": faker.last_name(),
        "SEGUNDO_APELLIDO_BENEFICIARIO": faker.last_name() if random.random() > 0.2 else '',
        "PAIS_DESTINO": faker.country()
    }
    beneficiarios_aleatorios.append(beneficiario)

# --- Unimos ambos listados ---
beneficiarios = beneficiarios_basados + beneficiarios_aleatorios

# ---------------------------------------------------------------------------
# 5. Codificación de columnas en `agentes_c_con_municipios`
# ---------------------------------------------------------------------------
# Este bloque reasigna códigos reales a etiquetas anónimas ("Sujeto_ObligadoX",
# "AgenteY") para facilitar la generación de nombres de fichero.

if 'ENT_COD_PAR_ENT' in agentes_c_con_municipios.columns:
    unique_sujetos = agentes_c_con_municipios['ENT_COD_PAR_ENT'].unique()
    sujeto_mapping = {s: f'Sujeto_Obligado{i+1}' for i, s in enumerate(unique_sujetos)}
    agentes_c_con_municipios['ENT_COD_PAR_ENT'] = agentes_c_con_municipios['ENT_COD_PAR_ENT'].map(sujeto_mapping)

if 'ENT_NAT_REF_COD' in agentes_c_con_municipios.columns:
    unique_agentes = agentes_c_con_municipios['ENT_NAT_REF_COD'].unique()
    agente_mapping = {a: f'Agente{i+1}' for i, a in enumerate(unique_agentes)}
    agentes_c_con_municipios['ENT_NAT_REF_COD'] = agentes_c_con_municipios['ENT_NAT_REF_COD'].map(agente_mapping)

# ---------------------------------------------------------------------------
# 6. Distribución de localidades y selección de combinaciones
# ---------------------------------------------------------------------------

distribucion_localidades = (
    agentes_c_con_municipios
    .groupby(['ENT_TOW_CIT_RES', 'ENT_COD_PAR_ENT', 'ENT_NAT_REF_COD'])
    .size()
    .reset_index(name='counts')
)
total_localidades = distribucion_localidades['counts'].sum()
distribucion_localidades['proporcion'] = distribucion_localidades['counts'] / total_localidades

# Seleccionamos una combinación (ciudad + sujeto + agente) para cada fichero
combinaciones_seleccionadas = random.choices(
    distribucion_localidades.index,
    weights=distribucion_localidades['proporcion'],
    k=num_ficheros
)

# Mapeo agente → cliente designado (quien actuará como "agente" en los CSV)
designated_agents = {ag: random.choice(clientes_ordenantes) for ag in agente_mapping.values()}

# ---------------------------------------------------------------------------
# 7. Bucle de generación de cada fichero CSV
# ---------------------------------------------------------------------------

for i, fichero in enumerate(ficheros):
    # --------- Parámetros principales de este archivo ---------
    num_transacciones = random.randint(300, 1200)
    min_importe, max_importe = 50, 3000  # rango de importes en euros

    # ----------------- Fechas y horas -----------------
    fechas_aleatorias = []
    horas_aleatorias = []
    for _ in range(num_transacciones):
        fecha = datetime(2023, random.randint(1, 12), random.randint(1, 28))
        fechas_aleatorias.append(fecha.strftime('%d/%m/%Y'))
        # 90 % en horario de oficina, 10 % aleatorio
        if random.random() < 0.9:
            hora_str = f"{random.randint(9, 21):02d}:{random.randint(0, 59):02d}"
        else:
            hora_str_full = faker.time()
            try:
                hora_str = datetime.strptime(hora_str_full, "%H:%M:%S").strftime("%H:%M")
            except ValueError as e:
                print(f"Error parsing time string: {e}")
                hora_str = "00:00"
        horas_aleatorias.append(hora_str)

    fechas_aleatorias.sort()  # opcional: ordenamos las fechas
    importes_aleatorios = [random.randint(min_importe, max_importe) for _ in range(num_transacciones)]

    def generar_numero_transaccion():
        """Crea un identificador único con prefijo TXN-"""
        return 'TXN-' + ''.join(random.choices('ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789', k=8))

    # ---------------- Estado de operación ----------------
    estados_operacion = random.choices(
        ['EXITOSA', 'FALLIDA', 'CANCELADA'],
        weights=[0.9, 0.04, 0.06],
        k=num_transacciones
    )

    # ---------------- Ordenantes y beneficiarios ----------------
    ordenantes_seleccionados = random.choices(clientes_ordenantes, k=num_transacciones)

    # --- Forzamos que la primera transacción pertenezca al agente designado -----
    index_seleccionado = combinaciones_seleccionadas[i]
    agente_del_fichero = distribucion_localidades.loc[index_seleccionado, 'ENT_NAT_REF_COD']
    cliente_agente = designated_agents[agente_del_fichero]
    doc_agente_designado = cliente_agente["NUM_DOC_ORDENANTE"]
    if not any(o["NUM_DOC_ORDENANTE"] == doc_agente_designado for o in ordenantes_seleccionados):
        ordenantes_seleccionados[0] = cliente_agente

    # ----- Selección de beneficiarios alineados con los ordenantes -----
    beneficiarios_seleccionados = []
    for ordenante in ordenantes_seleccionados:
        beneficiarios_posibles = [
            b for b in beneficiarios
            if b['PAIS_DESTINO'] == ordenante['PAIS_DOC_ORDENANTE'] and (
                b['APELLIDO_BENEFICIARIO'] == ordenante['APELLIDO_ORDENANTE'] or
                b['APELLIDO_BENEFICIARIO'] == ordenante['SEGUNDO_APELLIDO_ORDENANTE']
            )
        ]
        beneficiarios_seleccionados.append(random.choice(beneficiarios_posibles) if beneficiarios_posibles else random.choice(beneficiarios))

    # -------------------------------------------------------------------
    # 8. Construcción del DataFrame base con todas las transacciones
    # -------------------------------------------------------------------

    data = {
        "NUMERO_TRANSACCION": [generar_numero_transaccion() for _ in range(num_transacciones)],
        "FECHA": fechas_aleatorias,
        "HORA": horas_aleatorias,
        "IMPORTE": importes_aleatorios,
        "ESTADO_OPERACION": estados_operacion,
        "PAIS_ORIGEN": ['España'] * num_transacciones,
    }

    # --- Inyectamos datos de ordenantes y beneficiarios (campo a campo) ---
    for j, ordenante in enumerate(ordenantes_seleccionados):
        for key, value in ordenante.items():
            data.setdefault(key, [None] * num_transacciones)
            data[key][j] = value

    for j, beneficiario in enumerate(beneficiarios_seleccionados):
        for key, value in beneficiario.items():
            data.setdefault(key, [None] * num_transacciones)
            data[key][j] = value
        # Aseguramos que el flag es_Agente existe (por si falta)
        ordenante.setdefault("es_Agente", False)

    # Asignamos ciudad, sujeto y agente (las mismas para todo el fichero)
    data['ENT_TOW_CIT_RES'] = [distribucion_localidades.loc[index_seleccionado, 'ENT_TOW_CIT_RES']] * num_transacciones
    data['ENT_COD_PAR_ENT'] = [distribucion_localidades.loc[index_seleccionado, 'ENT_COD_PAR_ENT']] * num_transacciones
    data['ENT_NAT_REF_COD'] = [agente_del_fichero] * num_transacciones

    # Flag es_Agente: True si coincide con el agente designado
    data["es_Agente"] = [o["NUM_DOC_ORDENANTE"] == doc_agente_designado for o in ordenantes_seleccionados]

    # --- DataFrame inicial ---
    df = pd.DataFrame(data)

    # -------------------------------------------------------------------
    # 9. Inyección de patrones de riesgo (cuckoo smurfing, fraccionamiento…)
    # -------------------------------------------------------------------
    # 

    # 9.1 Tramas concentradas y distribuidas ---------------------------------------------------
    agentes_unicos = df["ENT_NAT_REF_COD"].unique().tolist()
    num_agentes = len(agentes_unicos)

    p_concentrado = 0.10            # 10 % agentes con trama concentrada
    p_distribuido = 0.10            # otros 10 % en clústeres distribuidos

    num_concentrado = max(1, int(num_agentes * p_concentrado))
    num_distribuido_clusters = max(1, int(num_agentes * p_distribuido))

    agentes_concentrado = random.sample(agentes_unicos, num_concentrado)
    agentes_restantes = [a for a in agentes_unicos if a not in agentes_concentrado]
    random.shuffle(agentes_restantes)

    cluster_size = max(2, len(agentes_restantes) // num_distribuido_clusters)
    clusters_distribuido = [
        agentes_restantes[i*cluster_size:(i+1)*cluster_size]
        for i in range(num_distribuido_clusters)
        if agentes_restantes[i*cluster_size:(i+1)*cluster_size]
    ]

    # -- Inyección de transacciones smurfing concentradas --
    for agente in agentes_concentrado:
        beneficiario_central = {
            "NOMBRE_BENEFICIARIO": "Central",
            "APELLIDO_BENEFICIARIO": "Beneficiario",
            "SEGUNDO_APELLIDO_BENEFICIARIO": "",
            "PAIS_DESTINO": random.choice(["España", "Francia", "Italia"])
        }
        num_transacciones_conc = random.randint(2, 5)
        ordenantes_agente = [o for o in ordenantes_seleccionados if o.get("ENT_NAT_REF_COD") == agente] or ordenantes_seleccionados
        for _ in range(num_transacciones_conc):
            ordenante = random.choice(ordenantes_agente)
            df = df.append({
                "NUMERO_TRANSACCION": generar_numero_transaccion(),
                "FECHA": random.choice(fechas_aleatorias),
                "HORA": random.choice(horas_aleatorias),
                "IMPORTE": random.randint(10, 200),
                "ESTADO_OPERACION": "EXITOSA",
                "PAIS_ORIGEN": "España",
                "NOMBRE_ORDENANTE": ordenante["NOMBRE_ORDENANTE"],
                "APELLIDO_ORDENANTE": ordenante["APELLIDO_ORDENANTE"],
                "SEGUNDO_APELLIDO_ORDENANTE": ordenante.get("SEGUNDO_APELLIDO_ORDENANTE", ""),
                "PAIS_DOC_ORDENANTE": ordenante["PAIS_DOC_ORDENANTE"],
                "NUM_DOC_ORDENANTE": ordenante["NUM_DOC_ORDENANTE"],
                "PAIS_NAC_ORDENANTE": ordenante["PAIS_NAC_ORDENANTE"],
                "FECHA_NAC_ORDENANTE": ordenante["FECHA_NAC_ORDENANTE"],
                "es_Agente": False,
                "NOMBRE_BENEFICIARIO": beneficiario_central["NOMBRE_BENEFICIARIO"],
                "APELLIDO_BENEFICIARIO": beneficiario_central["APELLIDO_BENEFICIARIO"],
                "SEGUNDO_APELLIDO_BENEFICIARIO": beneficiario_central["SEGUNDO_APELLIDO_BENEFICIARIO"],
                "PAIS_DESTINO": beneficiario_central["PAIS_DESTINO"],
                "ENT_TOW_CIT_RES": distribucion_localidades.loc[index_seleccionado, 'ENT_TOW_CIT_RES'],
                "ENT_COD_PAR_ENT": distribucion_localidades.loc[index_seleccionado, 'ENT_COD_PAR_ENT'],
                "ENT_NAT_REF_COD": agente,
            }, ignore_index=True)

    # -- Inyección de transacciones smurfing distribuidas --
    for cluster in clusters_distribuido:
        beneficiario_distribuido = {
            "NOMBRE_BENEFICIARIO": "Distribuido",
            "APELLIDO_BENEFICIARIO": "Beneficiario",
            "SEGUNDO_APELLIDO_BENEFICIARIO": "",
            "PAIS_DESTINO": random.choice(["España", "Francia", "Italia"])
        }
        num_transacciones_cluster = random.randint(1, 3)
        for agente in cluster:
            ordenantes_agente = [o for o in ordenantes_seleccionados if o.get("ENT_NAT_REF_COD") == agente] or ordenantes_seleccionados
            for _ in range(num_transacciones_cluster):
                ordenante = random.choice(ordenantes_agente)
                df = df.append({
                    "NUMERO_TRANSACCION": generar_numero_transaccion(),
                    "FECHA": random.choice(fechas_aleatorias),
                    "HORA": random.choice(horas_aleatorias),
                    "IMPORTE": random.randint(10, 200),
                    "ESTADO_OPERACION": "EXITOSA",
                    "PAIS_ORIGEN": "España",
                    "NOMBRE_ORDENANTE": ordenante["NOMBRE_ORDENANTE"],
                    "APELLIDO_ORDENANTE": ordenante["APELLIDO_ORDENANTE"],
                    "SEGUNDO_APELLIDO_ORDENANTE": ordenante.get("SEGUNDO_APELLIDO_ORDENANTE", ""),
                    "PAIS_DOC_ORDENANTE": ordenante["PAIS_DOC_ORDENANTE"],
                    "NUM_DOC_ORDENANTE": ordenante["NUM_DOC_ORDENANTE"],
                    "PAIS_NAC_ORDENANTE": ordenante["PAIS_NAC_ORDENANTE"],
                    "FECHA_NAC_ORDENANTE": ordenante["FECHA_NAC_ORDENANTE"],
                    "es_Agente": False,
                    "NOMBRE_BENEFICIARIO": beneficiario_distribuido["NOMBRE_BENEFICIARIO"],
                    "APELLIDO_BENEFICIARIO": beneficiario_distribuido["APELLIDO_BENEFICIARIO"],
                    "SEGUNDO_APELLIDO_BENEFICIARIO": beneficiario_distribuido["SEGUNDO_APELLIDO_BENEFICIARIO"],
                    "PAIS_DESTINO": beneficiario_distribuido["PAIS_DESTINO"],
                    "ENT_TOW_CIT_RES": distribucion_localidades.loc[index_seleccionado, 'ENT_TOW_CIT_RES'],
                    "ENT_COD_PAR_ENT": distribucion_localidades.loc[index_seleccionado, 'ENT_COD_PAR_ENT'],
                    "ENT_NAT_REF_COD": agente,
                }, ignore_index=True)

    # 9.2 Otros indicadores de riesgo ----------------------------------------------------------
    # (A) Concentra 70 % de transacciones de riesgo en el 20 % de agentes
    total_trans = len(df)
    num_riesgo = max(1, int(total_trans * random.uniform(0.05, 0.10)))
    agentes_unicos = df["ENT_NAT_REF_COD"].unique().tolist()
    random.shuffle(agentes_unicos)
    top_count = max(1, int(len(agentes_unicos) * 0.2))
    top_agentes, otros_agentes = agentes_unicos[:top_count], agentes_unicos[top_count:]
    indices_top = df.index[df["ENT_NAT_REF_COD"].isin(top_agentes)].tolist()
    indices_otros = df.index[df["ENT_NAT_REF_COD"].isin(otros_agentes)].tolist()
    num_riesgo_top = min(len(indices_top), int(num_riesgo * 0.7))
    num_riesgo_otros = min(len(indices_otros), num_riesgo - num_riesgo_top)
    indices_riesgo = random.sample(indices_top, num_riesgo_top) + random.sample(indices_otros, num_riesgo_otros)

    # (B) Documentos repetidos con nombre distinto ---------------------------------------------
    for idx in indices_riesgo:
        df.loc[idx, ["NOMBRE_ORDENANTE", "APELLIDO_ORDENANTE"]] = [faker.first_name(), faker.last_name()]
        df.loc[idx, "SEGUNDO_APELLIDO_ORDENANTE"] = faker.last_name() if random.random() > 0.2 else ''

    # (C) Documentos erróneos ------------------------------------------------------------------
    def validoDNI(dni: str) -> bool:
        tabla = "TRWAGMYFPDXBNJZSQVHLCKE"; dig_ext = "XYZ"; reemp = {'X': '0', 'Y': '1', 'Z': '2'}
        dni = dni.upper()
        if len(dni) == 9:
            num, dig_control = dni[:8], dni[8]
            num = num.replace(num[0], reemp.get(num[0], num[0])) if num[0] in dig_ext else num
            return num.isdigit() and tabla[int(num) % 23] == dig_control
        return False

    num_riesgo_doc_err = max(0, int(total_trans * random.uniform(0.01, 0.02)))
    indices_riesgo_doc_err = random.sample(list(df.index), num_riesgo_doc_err)
    for idx in indices_riesgo_doc_err:
        df.loc[idx, "NUM_DOC_ORDENANTE"] = "ZZZ" + ''.join(random.choices("0123456789ABCDEF", k=5))

    # (D) Fraccionamiento (> 3 000 €) ---------------------------------------------------------
    num_riesgo_fracc = max(0, int(total_trans * 0.01))
    indices_fracc = random.sample(list(df.index), num_riesgo_fracc)
    for idx in indices_fracc:
        base_imp = df.loc[idx, "IMPORTE"]
        if base_imp < 3000:
            nuevos_importes = [random.randint(800, 1500) for _ in range(3)]
            if sum(nuevos_importes) < 3001:
                nuevos_importes[0] += 3001 - sum(nuevos_importes)
            for imp in nuevos_importes:
                nueva_fila = df.loc[idx].copy()
                nueva_fila["IMPORTE"] = imp
                nueva_fila["NUMERO_TRANSACCION"] = "TXN-FRAC-" + ''.join(random.choices('ABC123', k=5))
                df = df.append(nueva_fila, ignore_index=True)
            df.drop(idx, inplace=True)

    # (E) Envíos repetidos a un mismo beneficiario (> 10 000 € / mes) -------------------------
    num_riesgo_benef = max(0, int(total_trans * 0.012))
    indices_riesgo_benef = random.sample(list(df.index), num_riesgo_benef)
    for idx in indices_riesgo_benef:
        random_month, random_day = random.randint(1, 12), random.randint(1, 28)
        fecha_fija = f"{random_day:02d}/{random_month:02d}/2023"
        suma, transacciones_extra = 0, []
        while suma < 10001:
            imp = random.randint(2000, 3000)
            suma += imp
            nueva_fila = df.loc[idx].copy()
            nueva_fila.update({
                "IMPORTE": imp,
                "NUMERO_TRANSACCION": "TXN-HIGH-" + ''.join(random.choices('XYZ987', k=5)),
                "FECHA": fecha_fija,
            })
            transacciones_extra.append(nueva_fila)
        df = pd.concat([df.drop(idx), pd.DataFrame(transacciones_extra)], ignore_index=True)

    # -------------------------------------------------------------------
    # 10. Etiquetas PEP (personas políticamente expuestas)
    # -------------------------------------------------------------------

    df['es_PEP'] = False
    pep_idx = np.random.choice(df.index, size=max(1, int(len(df) * 0.005)), replace=False)
    df.loc[pep_idx, 'es_PEP'] = True

    # -------------------------------------------------------------------
    # 11. Aseguramos que el agente designado aparezca marcado como tal
    # -------------------------------------------------------------------

    codigo_agente = df["ENT_NAT_REF_COD"].iloc[0]
    agente_designado = designated_agents[codigo_agente]
    if not any(o["NUM_DOC_ORDENANTE"] == agente_designado["NUM_DOC_ORDENANTE"] for o in ordenantes_seleccionados):
        ordenantes_seleccionados[0] = agente_designado
        for key, value in agente_designado.items():
            df.at[0, key] = value
    df.loc[df['NUM_DOC_ORDENANTE'] == agente_designado["NUM_DOC_ORDENANTE"], "es_Agente"] = True

    # -------------------------------------------------------------------
    # 12. Serialización a CSV
    # -------------------------------------------------------------------
    sujeto_obligado_nombre = df['ENT_COD_PAR_ENT'].iloc[0].replace(" ", "_") if 'ENT_COD_PAR_ENT' in df.columns else "Desconocido"
    agente_nombre = df['ENT_NAT_REF_COD'].iloc[0].replace(" ", "_") if 'ENT_NAT_REF_COD' in df.columns else "Desconocido"
    nombre_fichero = f"{sujeto_obligado_nombre}__{agente_nombre}.csv"
    df.to_csv(nombre_fichero, index=False, encoding='utf-8-sig')
    print(f"Fichero {nombre_fichero} generado con {num_transacciones} transacciones.")

print("Todos los ficheros CSV han sido generados.")


Introduce el número de ficheros CSV a generar: 50
Fichero Sujeto_Obligado8__Agente540.csv generado con 1045 transacciones.
Fichero Sujeto_Obligado5__Agente7779.csv generado con 611 transacciones.
Fichero Sujeto_Obligado13__Agente14026.csv generado con 856 transacciones.
Fichero Sujeto_Obligado10__Agente437.csv generado con 705 transacciones.
Fichero Sujeto_Obligado10__Agente2427.csv generado con 429 transacciones.
Fichero Sujeto_Obligado3__Agente6623.csv generado con 593 transacciones.
Fichero Sujeto_Obligado8__Agente6393.csv generado con 717 transacciones.
Fichero Sujeto_Obligado8__Agente3285.csv generado con 820 transacciones.
Fichero Sujeto_Obligado12__Agente1108.csv generado con 773 transacciones.
Fichero Sujeto_Obligado12__Agente2264.csv generado con 931 transacciones.
Fichero Sujeto_Obligado3__Agente9519.csv generado con 698 transacciones.
Fichero Sujeto_Obligado8__Agente7391.csv generado con 1160 transacciones.
Fichero Sujeto_Obligado6__Agente959.csv generado con 833 transaccion